<a href="https://colab.research.google.com/github/furkanoruc/IntroDL/blob/main/RNN-LSTM-imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### In this homework, to debug my code, I have benefited from the following notebook: https://github.com/rasbt/stat453-deep-learning-ss21/blob/main/L15/1_lstm.ipynb



In [1]:
!pip install -U torch==1.8.0 torchtext==0.9.0

     |████████████████████████████████| 735.5 MB 14 kB/s 
     |████████████████████████████████| 7.1 MB 36.5 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.8.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.8.0 which is incompatible.


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd
import spacy

# Import Required Libraries & Data Loading

In [4]:
#importing the training data
df=pd.read_csv('IMDB Dataset.csv')
print(df.shape)
df.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


# Data Preparation

In [5]:
"""
sentiment : 0 = negative, 1 = positive 
use the following to get the sentiment of a sentence :  
sentiment = 0 if sentiment is negative else 1


use np.where to get the sentiment of a sentence :
"""
df['sentiment'] = np.where(df['sentiment'] == 'positive', 1, 0)

In [6]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [7]:
df.columns = ['TEXT_COLUMN_NAME', 'LABEL_COLUMN_NAME']

In [8]:
"""
Load the spacy model and load the English language model from https://spacy.io/usage/models
"""
import spacy
nlp = spacy.load("en_core_web_sm")

# Based on provided website

In [9]:
# general Settings

RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.01
BATCH_SIZE = 64
NUM_EPOCHS = 5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_CLASSES = 2

# Text & label Preparation

In [10]:
# Define feature processing
"""
Define the fields for the data.
"""
TEXT = torchtext.legacy.data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm')

In [11]:
# Define Label processing
LABEL = torchtext.legacy.data.LabelField(dtype = torch.long)

In [12]:
"""
Define the fields for the data.
"""

df.to_csv('moviedata.csv', index = None)
df = pd.read_csv('moviedata.csv')
df.head()

,TEXT_COLUMN_NAME,LABEL_COLUMN_NAME
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [13]:
# process the dataset

fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
                    path = 'moviedata.csv', ### ADD YOUR DATASET PATH HERE ###
                    format = 'csv', ### ADD YOUR DATASET FORMAT HERE ###
                    skip_header = True, ### ADD YOUR SKIP HEADER HERE ### 
                    fields = fields ### ADD YOUR FIELDS HERE ### 
)

# Data Split

In [14]:
# Split dataset into train and test set

train_data, test_data = dataset.split(split_ratio = [0.8, 0.2], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of test data', len(test_data))

Length of train data 40000
Length of test data 10000


In [15]:
train_data, val_data = train_data.split(split_ratio = [0.85, 0.15], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of valid data', len(val_data))

Length of train data 34000
Length of valid data 6000


# Data Observation after Tokenization

In [16]:
# Look at first traning example

print(vars(train_data.examples[2000]))

{'TEXT_COLUMN_NAME': ['Flipping', 'through', 'the', 'channels', 'I', 'was', 'lucky', 'enough', 'to', 'stumble', 'upon', 'the', 'beginning', 'of', 'this', 'movie', '.', 'I', 'must', 'admit', 'that', 'it', 'grabbed', 'my', 'attention', 'almost', 'immediately', '.', 'I', 'love', 'older', 'films', 'and', 'this', 'is', 'or', 'should', 'be', 'considered', 'a', 'classic', '!', 'One', 'of', 'the', 'most', 'wonderful', 'rarities', 'of', 'this', 'movie', 'is', 'that', 'the', 'main', 'character', 'was', 'not', 'only', 'female', 'but', 'she', 'was', 'also', 'a', 'bad', 'girl', '.', 'I', 'highly', 'recommend', 'this', 'movie', '!'], 'LABEL_COLUMN_NAME': '1'}


In [17]:
# Build Vocabulary

TEXT.build_vocab(train_data, max_size = VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'vocabulary size: {len(TEXT.vocab)}')
print(f'Label Size: {len(LABEL.vocab)}')

vocabulary size: 20002
Label Size: 2


 2 extra value in vocabulary is because added (unknown) and (padding)

In [18]:
# Print the most common words: Use the most_common method of the TEXT vocabulary
print(TEXT.vocab.freqs.most_common(5))

[('the', 390972), (',', 369444), ('.', 318719), ('a', 210502), ('and', 210006)]


In [19]:
# Token corresponding to first 10 Indices

print(TEXT.vocab.itos[:10]) #itos = Integer to string

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


# Data Preparation for Batch wise Implimentation

In [20]:
# Define Dataloader

train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
        (train_data, val_data, test_data), ### ADD YOUR SPLIT DATA HERE (Make sure you add it in a tuple) ###
        batch_size = 64, ### ADD YOUR BATCH SIZE HERE ###
        sort_within_batch = False, ### ADD YOUR SORT WITHIN BATCH HERE ### 
        sort_key = lambda x : len(x.TEXT_COLUMN_NAME), 
        device = DEVICE
    )

In [21]:
# Testing the iterators (note that the number of rows depends on the longest document in the respective batch):

print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

Train
Text matrix size: torch.Size([1039, 64])
Target vector size: torch.Size([64])

Valid:
Text matrix size: torch.Size([50, 64])
Target vector size: torch.Size([64])

Test:
Text matrix size: torch.Size([44, 64])
Target vector size: torch.Size([64])


# Model Building

In [23]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        ### ADD YOUR CODE HERE ###
        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        self.rnn = torch.nn.LSTM(embedding_dim, hidden_dim)        
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        
        ### END YOUR CODE ### 

    def forward(self, text):
        ### ADD YOUR CODE HERE ###
        # text dim: [sentence length, batch size]
        embedded = self.embedding(text)
        
        # embedded dim: [sentence length, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded)
        
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]
        hidden.squeeze_(0)

        # hidden dim: [batch size, hidden dim]

        output = self.fc(hidden)
        ### END YOUR CODE ###
        return output

In [24]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim= len(TEXT.vocab), ### ADD YOUR INPUT DIM HERE. This can be the length of your vocabulary or the embedding dim ###
            embedding_dim= EMBEDDING_DIM, ### ADD YOUR EMBEDDING DIM HERE ###
            hidden_dim= HIDDEN_DIM, ### ADD YOUR HIDDEN DIM HERE ###
            output_dim= NUM_CLASSES ### ADD NUMBER OF CLASSES HERE ###
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr= LEARNING_RATE) ### ADD YOUR OPTIMIZER HERE ###

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 44.1 MB/s eta 0:00:27tcmalloc: large alloc 1147494400 bytes == 0x3aa04000 @  0x7f0bade86615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████               | 1055.7 MB 59.9 MB/s eta 0:00:16tcmalloc: large alloc 1434370048 bytes == 0x7f05a000 @  0x7f0bade86615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████▋          | 1336.2 MB 1.5 MB/s eta 0:07:01tcmalloc: large alloc 1792966656 bytes

# Define Accuracy

In [25]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

# Model Run

In [26]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/005 | Batch 000/532 | Loss: 0.7094
Epoch: 001/005 | Batch 050/532 | Loss: 0.7088
Epoch: 001/005 | Batch 100/532 | Loss: 0.6939
Epoch: 001/005 | Batch 150/532 | Loss: 0.6937
Epoch: 001/005 | Batch 200/532 | Loss: 0.6955
Epoch: 001/005 | Batch 250/532 | Loss: 0.6916
Epoch: 001/005 | Batch 300/532 | Loss: 0.6972
Epoch: 001/005 | Batch 350/532 | Loss: 0.6882
Epoch: 001/005 | Batch 400/532 | Loss: 0.6923
Epoch: 001/005 | Batch 450/532 | Loss: 0.6950
Epoch: 001/005 | Batch 500/532 | Loss: 0.6912
training accuracy: 50.28%
valid accuracy: 50.45%
Time elapsed: 3.03 min
Epoch: 002/005 | Batch 000/532 | Loss: 0.6914
Epoch: 002/005 | Batch 050/532 | Loss: 0.6959
Epoch: 002/005 | Batch 100/532 | Loss: 0.6916
Epoch: 002/005 | Batch 150/532 | Loss: 0.6918
Epoch: 002/005 | Batch 200/532 | Loss: 0.6934
Epoch: 002/005 | Batch 250/532 | Loss: 0.6922
Epoch: 002/005 | Batch 300/532 | Loss: 0.6943
Epoch: 002/005 | Batch 350/532 | Loss: 0.6928
Epoch: 002/005 | Batch 400/532 | Loss: 0.6974
Epoch: 0

# Model Testing

In [27]:
import spacy
nlp = spacy.blank("en")

def predict_sentiment(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    return prediction[0][1].item()

print('Probability positive:')
predict_sentiment(model, "This is such an awesome movie, I really love it!")

Probability positive:


0.8662715554237366

In [28]:
print('Probability positive:')
predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")

Probability positive:


0.49010297656059265